<a href="https://colab.research.google.com/github/kaushalkuma-r/Machine-Learning-Course/blob/main/ML_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import tools

In [45]:
import numpy as np
import pandas as pd

## Get the data

In [46]:
visitors_df=pd.read_csv("https://raw.githubusercontent.com/kaushalkuma-r/Machine-Learning-Course/main/visitors-covid.csv")
covid_df=pd.read_csv("https://raw.githubusercontent.com/kaushalkuma-r/Machine-Learning-Course/main/covid-data.csv")

In [47]:
covid_df['date']=pd.to_datetime(covid_df['date'])
visitors_df['Day']=pd.to_datetime(visitors_df['Day'])
covid_df=covid_df[['iso_code', 'continent', 'location', 'date','new_cases']]
covid_df=covid_df[covid_df['location']=='India']
visitors_df=visitors_df[visitors_df['Entity']=='India']
visitors_df.rename(columns={'Day':'date'},inplace=True)
covid_merged=pd.merge(covid_df, visitors_df, on='date')
final_df=covid_merged.drop(columns=['iso_code', 'continent', 'location', 'date','Entity','Code'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [48]:
final_df

,new_cases,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,0.0,0.667,1.667,0.000,2.000,3.000,3.000
1,0.0,0.500,1.750,0.000,2.000,3.250,3.000
2,0.0,0.400,1.800,0.200,1.800,2.800,3.200
3,0.0,0.500,2.000,0.000,2.333,3.167,3.333
4,0.0,-0.143,1.714,0.714,1.429,3.571,0.143
...,...,...,...,...,...,...,...
466,173790.0,-61.714,-25.000,24.143,-49.143,-41.000,-45.429
467,165553.0,-61.286,-24.429,23.714,-48.714,-40.000,-44.571
468,152734.0,-61.143,-24.714,23.714,-49.000,-39.143,-44.286
469,127510.0,-60.143,-23.429,23.286,-48.286,-38.000,-43.429


In [49]:
data = final_df
data.head(5)

,new_cases,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,0.0,0.667,1.667,0.000,2.000,3.000,3.000
1,0.0,0.500,1.750,0.000,2.000,3.250,3.000
2,0.0,0.400,1.800,0.200,1.800,2.800,3.200
3,0.0,0.500,2.000,0.000,2.333,3.167,3.333
4,0.0,-0.143,1.714,0.714,1.429,3.571,0.143


## Train-Test split

In [50]:
shuffle_df = final_df.sample(frac=1)

# Defining a size for train set 
train_size = int(0.8 * len(final_df))

# Splitting dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

X_train , Y_train = train_set.drop('new_cases' , axis = 1).values , train_set['new_cases'].values.reshape(-1,1)
X_test , Y_test = test_set.drop('new_cases' , axis = 1).values , test_set['new_cases'].values.reshape(-1,1)

# print(X_train[:5])
# print(Y_train[:5])
# print(X_test[:5])
# print(Y_test[:5])

# Updated CART Decision Tree Model for Regression using Residual Sum of Squares

### Node class for CART

In [51]:
class Tree_Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

### Tree class for CART

In [52]:
class CART_DTR():
    def __init__(self, min_samples_split=2, max_depth=2):
              
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
               
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Tree_Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Tree_Node(value=leaf_value)

    def average(self,col):
      #Function to calculate the possible mean values of a Column
      l1=[]
      for i in range(len(col)-1):
        l1.append((col[i]+col[i+1])/2)
      return l1

    def get_best_split(self, dataset, num_samples, num_features):
              
        # dictionary to store the best split
        best_split = {}
        min_rss = +float("inf")
        # print(num_features)
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            # calling to get the mean of two consecutive elements of a feature.
            possible_thresholds_list=self.average(feature_values.tolist())
            possible_thresholds = np.array(possible_thresholds_list)
            # print(possible_thresholds)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute  residual sum of square for both left and right datasets.
                    rss_left = self.rss(left_y)
                    rss_right=self.rss(right_y)
                    curr_rss=rss_left+rss_right
                    # update the best split to get the minimum rss out of each posssible thresholds for each feature column.
                    if curr_rss<min_rss:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_rss
                        min_rss = curr_rss
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        # ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def calculate_leaf_value(self, Y):
        # ''' function to compute leaf node '''
        val = np.mean(Y)
        return val
    
    def fit(self, X, Y):
        # ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def helper_predict(self, x, tree):
        # helper function to traverse through the tree and return the predictions
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.helper_predict(x, tree.left)
        else:
            return self.helper_predict(x, tree.right)
    
    def predict(self, X):
    # Function to make predictions        
        predictions = [self.helper_predict(x, self.root) for x in X]
        return predictions
    def rss(self,X):
      # Function to calculate the rss 
        mean_data=sum(X)
        rss=0
        for i in X:
          rss=(mean_data-i)**2
        return rss

# ID3-A for Regression using Standard Deviation

In [53]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [54]:
class ID3_A_Regressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
               
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
            right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
            # creating the node for best split at this point in the tree
            return Node(best_split["feature_index"], best_split["threshold"], 
                        left_subtree, right_subtree, best_split["var_red"])
        
        # we are creating a leaf value for each Node, using the leaf_value we are creating our tree.
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    def average(self,col):
      #Function to calculate the possible mean values of a feature that will be used to calculate standard deviation reduction
      l1=[]
      for i in range(len(col)-1):
        l1.append((col[i]+col[i+1])/2)
      return l1

    def get_best_split(self, dataset, num_samples, num_features):
        # Function that will provide the best split for the data at an instant after calculating sdr for each feature and their possible mean values.
        best_split = {}
        max_var_red = -float("inf")
        # print(num_features)
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds_list=self.average(feature_values.tolist())
            possible_thresholds = np.array(possible_thresholds_list)
            # print(possible_thres)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.var_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
        return best_split
    
    def split(self, dataset, feature_index, threshold):       
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def var_reduction(self, parent, left, right):
        
        reduction = np.var(parent) - ((len(left) / len(parent)) * np.var(left) + (len(right) / len(parent)) * np.var(right))
        return reduction
    
    def calculate_leaf_value(self, Y):
        return  np.mean(Y)
                
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
                
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
            
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):      
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

## Fitting the model for CART

In [55]:
cart_regressor = CART_DTR(min_samples_split=3, max_depth=3)
cart_regressor.fit(X_train,Y_train)

### Fitting the model for ID3-A

In [56]:
id3_regressor = ID3_A_Regressor(min_samples_split=2, max_depth=4)
id3_regressor.fit(X_train,Y_train)

## Testing the created models and calculating their r2score.

In [57]:
Y_pred_id3 = id3_regressor.predict(X_test) 
Y_pred_cart = cart_regressor.predict(X_test) 

In [58]:
Y_test=[i[0] for i in Y_test.tolist()]

In [59]:
Y_test=np.array(Y_test)
Y_pred_id3=np.array(Y_pred_id3)

In [60]:
Y_pred_cart=np.array(Y_pred_cart)

In [61]:
def r2Score(y_pred,y_actual):
    sum_square_pred = np.sum((y_pred-y_actual)**2)
    sum_squares_actual = np.sum((y_actual- y_actual.mean())**2)
    score = (1- sum_square_pred/sum_squares_actual)
    return score

In [62]:
score_id3=r2Score(Y_pred_id3, Y_test)
score_id3

0.7715324116794547

In [63]:
score_cart=r2Score(Y_pred_cart, Y_test)
score_cart

0.6564599519073083

# K-Means Clustering for classifying the dataset

In [64]:
# Select any 3 random points and Make clusters on them
random_indices = np.random.choice(X_train.shape[0], size=3, replace=False)
centroids = X_train[random_indices, :]
print(centroids)

[[-60.    -12.857  16.286 -41.857 -51.143 -32.   ]
 [-56.857  -7.857  15.286 -39.286 -49.571 -32.143]
 [ -0.571   2.429   0.286   1.143   1.429   5.286]]


In [65]:
class Cluster_ID3():
    def __init__(self, centroids , k):
        self.root = {}
        # Initiate empty clusters
        self.clusters_train = {}
        self.clusters_label = {}
        self.centroid = centroids
        self.k = k

    def make_clusters(self , X , y, centroids):
        # Set the range for value of k (number of centroids)
        for i in range(self.k):
            self.clusters_train[i] = []
            self.clusters_label[i] = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            # Append the cluster of data to the dictionary
            self.clusters_train[euc_dist.index(min(euc_dist))].append(X[i])
            self.clusters_label[euc_dist.index(min(euc_dist))].append(y[i])

    def train(self):
        for i in range(self.k):
            temp = ID3_A_Regressor()
            temp.fit(X = self.clusters_train[i] , Y = self.clusters_label[i])
            self.root[i] = temp

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            idx = euc_dist.index(min(euc_dist))
            # print(idx)
            l = []
            l.append(X[i])
            y_pred.append(self.root[idx].predict(l))

        return y_pred

In [66]:
model = Cluster_ID3(centroids , k = 3)
model.make_clusters(X_train , Y_train , centroids)
model.train()

# Predicting new_cases using ID3-A after clustering the datapoints.

In [67]:
Y_pred_K_ID3 = model.predict(X_test)
print(Y_pred_K_ID3)

[[64188.936170212764], [390766.8], [16425.82], [244924.65], [19869.75], [64188.936170212764], [64188.936170212764], [3222.186440677966], [64188.936170212764], [23946.11111111111], [64188.936170212764], [49131.3], [3222.186440677966], [3222.186440677966], [64188.936170212764], [21051.8], [10.884615384615385], [23946.11111111111], [390766.8], [64188.936170212764], [19869.75], [64188.936170212764], [19869.75], [16425.82], [19869.75], [244924.65], [64188.936170212764], [64188.936170212764], [19869.75], [3222.186440677966], [64188.936170212764], [21051.8], [16425.82], [244924.65], [64188.936170212764], [19869.75], [21051.8], [23946.11111111111], [64188.936170212764], [16425.82], [345931.75], [49131.3], [10.884615384615385], [23946.11111111111], [10.884615384615385], [64188.936170212764], [3222.186440677966], [3222.186440677966], [21051.8], [3222.186440677966], [64188.936170212764], [16425.82], [64188.936170212764], [20934.0], [64188.936170212764], [64188.936170212764], [16425.82], [10.88461

In [68]:
Y_pred_K_ID3=[i[0] for i in Y_pred_K_ID3]

In [69]:
r2score_k_id3=r2Score(np.array(Y_pred_K_ID3), Y_test)
r2score_k_id3

0.9541387476641365

In [70]:
class Cluster_Cart():
    def __init__(self, centroids , k):
        self.root = {}
        # Initiate empty clusters
        self.clusters_train = {}
        self.clusters_label = {}
        self.centroid = centroids
        self.k = k

    def make_clusters(self , X , y, centroids):
        # Set the range for value of k (number of centroids)
        for i in range(self.k):
            self.clusters_train[i] = []
            self.clusters_label[i] = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            # Append the cluster of data to the dictionary
            self.clusters_train[euc_dist.index(min(euc_dist))].append(X[i])
            self.clusters_label[euc_dist.index(min(euc_dist))].append(y[i])

    def train(self):
        for i in range(self.k):
            temp = CART_DTR()
            temp.fit(self.clusters_train[i] , self.clusters_label[i])
            self.root[i] = temp

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            idx = euc_dist.index(min(euc_dist))
            # print(idx)
            l = []
            l.append(X[i])
            y_pred.append(self.root[idx].predict(l))

        return y_pred

In [71]:
model = Cluster_Cart(centroids , k = 3)
model.make_clusters(X_train , Y_train , centroids)
model.train()

# Predicting new_cases using updated CART model after clustering the datapoints.

In [72]:
Y_pred_K_Cart = model.predict(X_test)
print(Y_pred_K_Cart)

[[30216.441860465115], [243121.0], [32566.69230769231], [243121.0], [30216.441860465115], [30216.441860465115], [50657.8], [21619.2], [50657.8], [9276.25], [74438.22222222222], [29498.533333333333], [21619.2], [21619.2], [91532.06666666667], [16406.17391304348], [9276.25], [32566.69230769231], [375112.0], [50657.8], [30216.441860465115], [50657.8], [30216.441860465115], [20931.55], [30216.441860465115], [21619.2], [50657.8], [50657.8], [30216.441860465115], [21619.2], [30216.441860465115], [16406.17391304348], [85674.6], [375112.0], [68898.85], [30216.441860465115], [16406.17391304348], [16406.17391304348], [74438.22222222222], [22131.571428571428], [50657.8], [29498.533333333333], [9276.25], [9276.25], [9276.25], [74438.22222222222], [21619.2], [29498.533333333333], [16406.17391304348], [21619.2], [50657.8], [32566.69230769231], [68898.85], [29498.533333333333], [50657.8], [68898.85], [20931.55], [9276.25], [16406.17391304348], [30216.441860465115], [21619.2], [89259.2], [74438.222222

In [73]:
Y_pred_K_Cart=[i[0] for i in Y_pred_K_Cart]

In [74]:
r2score_k_cart=r2Score(np.array(Y_pred_K_Cart), Y_test)
r2score_k_cart

0.5540781190640378